# Local execution
Sometimes (often) you want to check how your pipeline is working locally before submitting it.

Here's the manual way of creating the pipeline and executing it.

At first, install the netunicorn-executor.

In [1]:
%pip install --upgrade netunicorn-executor

Note: you may need to restart the kernel to use updated packages.


If you do it in the already running asyncio loop (e.g., in Jupyter), you also need to install and import nest_asyncio. This is not required if you run it as a Python script.

In [2]:
%pip install nest-asyncio

Note: you may need to restart the kernel to use updated packages.


In [3]:
import nest_asyncio
nest_asyncio.apply()

Then, import and create an executor with any fake parameters.

In [4]:
from netunicorn.executor.executor import PipelineExecutor, ExecutorState

In [5]:
executor = PipelineExecutor(executor_id="", gateway_endpoint="fake", experiment_id="", heartbeat=False)

INFO:executor_Unknown:Parsed configuration: Gateway located on fake
INFO:executor_Unknown:Current directory: /home/kell/repos/netunicorn/examples


Create your pipeline. We'll use a simple one for that example.

In [6]:
from netunicorn.base import Pipeline, Task

In [7]:
class DummyTask(Task):
    def run(self):
        return 0

class DummyRepeaterTask(Task):
    def __init__(self, lookup_for: str):
        self.lookup_for = lookup_for
        super().__init__()

    def run(self):
        previous_results = self.previous_steps[self.lookup_for]
        return f"I've heard of {previous_results}!"

pipeline = Pipeline().then(DummyTask(name="dummy1")).then(DummyRepeaterTask(lookup_for='dummy1'))

Then, set `pipeline.report_results` to False (to prevent reporting results to our fake gateway), assign pipeline to executor.pipeline, and manually set the executor state to EXECUTING

In [8]:
pipeline.report_results = False
executor.pipeline = pipeline
executor.state = ExecutorState.EXECUTING

And then, call the executor.

In [9]:
executor()

INFO:executor_Unknown:Pipeline finished, start reporting results.
INFO:executor_Unknown:Skipping reporting results due to pipeline setting.


Now you can explore locally create log file and `executor.pipeline_results`.

In [10]:
print(executor.pipeline_results)

<Success: defaultdict(<class 'list'>, {'dummy1': [<Success: 0>], 'f44e8fe6-d54f-4418-bf5a-0e69fefcc63d': [<Success: I've heard of [<Success: 0>]!>]})>
